In [1]:
!pip install mlflow --quiet
import mlflow
print(mlflow.__version__)
!pip install pyspark

     |████████████████████████████████| 14.6 MB 117 kB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 180 kB 53.5 MB/s 
     |████████████████████████████████| 146 kB 55.8 MB/s 
     |████████████████████████████████| 596 kB 27.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
1.20.2
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8606423adb86e018fa5cc9a7edc63c0010cd8e3eea08b5887fcf0ef1e0b42417
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
!pip install pyngrok --quiet
from pyngrok import ngrok
from getpass import getpass
# Terminate open tunnels if any exist
# ngrok.kill()

     |████████████████████████████████| 745 kB 10.2 MB/s 


In [3]:
import pandas as pd
import pyspark
import mlflow
import mlflow.spark

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [18]:
sc.stop()

In [5]:
df = pd.read_csv('news_final (1).csv')

In [8]:
df = df.drop('author', axis=1)

In [9]:
train_df = spark.createDataFrame(df)

In [14]:
# Start a new MLflow run
import mlflow.mleap
with mlflow.start_run(run_name="Capstone_News") as run:
    # regular expression tokenizer
  regexTokenizer = RegexTokenizer(inputCol="summary", outputCol="words", pattern="\\W")
  # stop words
  add_stopwords = ["http","https","amp","rt","t","c","the"] 
  stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
  # bag of words count
  countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
  label_stringIdx = StringIndexer(inputCol = "topic", outputCol = "label")
  pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
  # Fit the pipeline to training documents.
  pipelineFit = pipeline.fit(train_df)
  dataset = pipelineFit.transform(train_df)
  dataset.show(5)
  # set seed for reproducibility
  (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
  print("Training Dataset Count: " + str(trainingData.count()))
  print("Test Dataset Count: " + str(testData.count()))
  lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
  lrModel = lr.fit(trainingData)
  predictions = lrModel.transform(testData)
  predictions.filter(predictions['prediction'] == 0) \
      .select("summary","topic","probability","label","prediction") \
      .orderBy("probability", ascending=False) \
      .show(n = 10, truncate = 30)

  evaluator_lr_cv = MulticlassClassificationEvaluator(predictionCol="prediction")
  evaluator_lr_cv.evaluate(predictions)
# pred=lr.predict(test_x)
  # (rmse, mae, r2) = eval_metrics(prediction,label_stringIdx)
  #  mlflow.log_metric("rmse", rmse)
  #  mlflow.log_metric("r2", r2)
  #  mlflow.log_metric("mae", mae)
  # mlflow.log_param("max_iter", param['max_iter'])
  # mlflow.log_param("penalty", param['penalty'])
  # mlflow.log_param("solver", param['solver'])
    
    #logging data which we used for this model
  # mlflow.log_artifact("96fa465011b74bf88df4babd69ad1401")

  # Log the model within the MLflow run
  # mlflow.spark.log_model(spark_model=lrModel, sample_input=df, artifact_path="lrModel")
  mlflow.spark.save_model(lrModel, "spark-model")
# mlflow.spark.load_model(path, run_id=None, dfs_tmpdir='/tmp/mlflow'
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

+----------+--------------------+--------------------+--------------------+-------------------+--------+--------------------+--------------------+--------------------+-----+
|Unnamed: 0|               title|             summary|                link|           pub_date|   topic|               words|            filtered|            features|label|
+----------+--------------------+--------------------+--------------------+-------------------+--------+--------------------+--------------------+--------------------+-----+
|         0|The difference be...|Business Analysis...|https://medium.co...|2021-10-16 06:09:58|    news|[business, analys...|[business, analys...|(304,[0,1,2,3,6,8...|  0.0|
|         1|Writing Off the E...|Starting a busine...|https://www.theba...|2021-10-22 20:46:00|    news|[starting, a, bus...|[starting, a, bus...|(304,[0,1,2,4,8,1...|  0.0|
|         2|How To Register a...|Finding a busines...|https://blog.hubs...|2021-10-19 11:00:00|business|[finding, a, busi...|[find

In [15]:
# Enter your auth token when the code is running
# 1zt2ytXyrAhXfMSFbjJBkqbCmD6_5FEDkfjsNCearfyE3i2JD
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken:')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken:··········


t=2021-10-23T09:59:42+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


MLflow Tracking UI: https://b624-35-233-249-132.ngrok.io
